# Importing the Libraries

In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading the dataset

In [257]:
# Load and prepare data
data = pd.read_csv('train.csv')
data = np.array(data) # We need just numpy array as we will be doing only algebra
m, n = data.shape # Lets store the shape into neat variables
np.random.shuffle(data) # Randomise the data before splitting into training and validation set

data_dev = data[0:1000].T # Splits the first 1000 digits into validation dataset and transpose it, as Neural Networks work with data for a value in column
Y_dev = data_dev[0] # All the labels(digits) are stored here
X_dev = data_dev[1:n] # All the pixel value are stored here
X_dev = X_dev / 255. # The pixel values are normalised to a activation value between 0 and 1 by dividing it with 255

In [258]:
# Same process done with the training dataset
data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_, m_train = X_train.shape

# Initializing the Parameters

In [259]:
# Initialize parameters
def init_params(): # Using a He initialization so that it avoids dying neurons
    W1 = np.random.randn(16, 784) * np.sqrt(2 / 784) 
    b1 = np.zeros((16, 1))
    W2 = np.random.randn(16, 16) * np.sqrt(2 / 16)
    b2 = np.zeros((16, 1))
    W3 = np.random.randn(10, 16) * np.sqrt(2 / 16)
    b3 = np.zeros((10, 1))
    return W1, b1, W2, b2, W3, b3

# Calculating the activation functions

In [260]:
# Activation functions
def ReLU(Z):
    return np.maximum(0, Z) # Gives the value Z if Z is some positive value, otherwise gives 0

def ReLU_deriv(Z):
    return Z > 0 # The derivative gives 1 if Z, as it will be set to True, otherwise gives 0(False).

def softmax(Z):
    A = np.exp(Z - np.max(Z, axis=0, keepdims=True)) # Provides a stability, the Temp value is set to the highest value Z can get
    return A / np.sum(A, axis=0, keepdims=True)

# Forward Propogation

In [261]:
# Forward propagation
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1 # Simpe calculation of all the Z and A values
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3) # Only the last layer uses a softmax function with a temp to give a normal distrubition of final activation values
    return Z1, A1, Z2, A2, Z3, A3

# Backward Propogation

In [262]:
# One-hot encoding
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1)) # Creates an array of zeros of 41000 rows and 10 columns(to store the value of digit in activation form) 
    one_hot_Y[np.arange(Y.size), Y] = 1 # sets the activation of a number to 1 based on the digit(if 2nd digit is 2, then column 2 and row 3 is set to 1)
    return one_hot_Y.T

In [263]:
# Backward propagation
def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    m = Y.size # All the mathematics is explained in the report. Please consult.
    one_hot_Y = one_hot(Y)

    dZ3 = A3 - one_hot_Y
    dW3 = (1 / m) * dZ3.dot(A2.T)
    db3 = (1 / m) * np.sum(dZ3, axis=1, keepdims=True)

    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = (1 / m) * dZ2.dot(A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = (1 / m) * dZ1.dot(X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2, dW3, db3

# Updating Parameters

In [264]:
# Update parameters
def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 -= alpha * dW1 # All the parameters are updated
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    W3 -= alpha * dW3
    b3 -= alpha * db3
    return W1, b1, W2, b2, W3, b3

# Calculating the accuracy

In [268]:
# Prediction and accuracy
def get_predictions(A3):
    return np.argmax(A3, axis=0) # Since a3 is (10, 41000), it returns the index of where in the matrix higeset probabilities are i.e the predicted digit. If index 2, then digit predicted is 2.

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y) # Checks to see how many predictions are same as true value by calculating mean

# Training loop
def gradient_descent(X, Y, alpha, iterations): # Alpha is set by user
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations): # Trained for a number of iterations set by user
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3,
                                               dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 100 == 0: # Prints the accuracy on every 100th iteration
            predictions = get_predictions(A3)
            acc = get_accuracy(predictions, Y)
            print(f"Iteration {i}: Accuracy = {acc:.4f}")
        elif i == 999:
            predictions = get_predictions(A3)
            acc = get_accuracy(predictions, Y)
            print(f"Last Iteration: Accuracy = {acc:.4f}") # Prints the final accuracy
    return W1, b1, W2, b2, W3, b3

In [269]:
# Train the network
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, alpha=0.1, iterations=1000)

Iteration 0: Accuracy = 0.0740
Iteration 100: Accuracy = 0.8611
Iteration 200: Accuracy = 0.8974
Iteration 300: Accuracy = 0.9097
Iteration 400: Accuracy = 0.9177
Iteration 500: Accuracy = 0.9234
Iteration 600: Accuracy = 0.9287
Iteration 700: Accuracy = 0.9323
Iteration 800: Accuracy = 0.9361
Iteration 900: Accuracy = 0.9391
Last Iteration: Accuracy = 0.9402
